Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [X] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [X] Continue to participate in our Kaggle challenge. 
- [X] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [X] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [X] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [5]:
import pandas as pd
import numpy as np

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [3]:
import pandas_profiling

profile_report = train.profile_report(
    check_correlation_pearson=False,
    correlations={
        'pearson': False,
        'spearman': False,
        'kendall': False,
        'phi_k': False,
        'cramers': False,
        'recoded': False,
    },
    plot={'histogram': {'bayesian_blocks_bins': False}},
)

profile_report

In [6]:
def preprocess(X):
    """
    Preprocess the data by changing datatypes for certain
    rows and handling data abnormalities by converting to
    NaN.
    """
    
    # Empty strings and unknowns should be treated as NaN
    nan_strings = ['', 'unknown', 'Unknown']
    X = X.replace(to_replace=nan_strings, value=np.nan)
    
    # Change date_recorded to datetime object
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Because region_code and district_code are categorical and not numeric data,
    # we should convert them to strings.
    X['region_code'] = X['region_code'].apply(str)
    X['district_code'] = X['district_code'].apply(str)
    
    # The public_meeting and permit columns are boolean, so they should
    # act as such in the dataframe.
    X['public_meeting'] = X['public_meeting'].astype(bool)
    X['permit'] = X['permit'].astype(bool)
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zeroes.
    near_zero = X['latitude'] > -1e-04
    X.loc[near_zero, 'latitude'] = 0
    
    near_zero = X['longitude'] < 1e-04
    X.loc[near_zero, 'longitude'] = 0
    
    # When columns have zeros and don't make sense, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    cols_with_zeros = ['amount_tsh', 'gps_height', 'longitude',
                       'latitude', 'population', 'construction_year']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
    
    # Convert the categorical columns to lowercase to account for possible
    # random capitalizations.
    categorical = X.select_dtypes(exclude=['number', 'bool', 'datetime']).columns.tolist()
    
    for feature in categorical:
        X[feature] = X[feature].str.lower()
        
    # Drop the following columns because they add nothing to the data for analysis
    # id - constantly varies, we can already ID by index
    # recorded_by - constant value, does not help predict status_group
    to_drop = ['id', 'recorded_by']
    X = X.drop(columns=to_drop)
    
    return X

train = preprocess(train)
test = preprocess(test)

In [7]:
def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # Create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['amount_tsh', 'gps_height', 'longitude',
                       'latitude', 'population', 'construction_year']
    for col in cols_with_zeros:
        X[col+'_MISSING'] = X[col].isnull()

    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    
    # Return the wrangled dataframe
    return X

train = wrangle(train)
test = wrangle(test)

In [8]:
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [9]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_test = test[features]

In [10]:
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from time import time

In [12]:
# Fit pipeline on GridSearchCV
pipe = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(n_estimators=1000,
                           max_features=10,
                           random_state=42,
                           n_jobs=-1)
)

# Fit the cross-validation on train
t0 = time()
pipe.fit(X_train, y_train)
print(f'Done in {(time()-t0):0.3f}s\n')

# Score on train, val
print('Train Accuracy:', pipe.score(X_train, y_train))

Done in 38.554s

Train Accuracy: 0.9976430976430977


In [45]:
import category_encoders as ce
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median'), 
    RandomForestClassifier(max_depth=None, n_estimators=100, random_state=42, n_jobs=-1)
)

param_distributions = {
    'randomforestclassifier__max_features': uniform(0,1),
    'randomforestclassifier__max_depth': [20, None]
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=12, 
    cv=4, 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 4 folds for each of 12 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   49.7s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  46 out of  48 | elapsed:  6.6min remaining:   17.0s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  6.6min finished


In [46]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation accuracy', search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.35909412855987566}
Cross-validation accuracy 0.8067171717171717


In [47]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,10,7,4,8,11,3,6,5,9,2,0,1
mean_fit_time,19.6558,29.5763,19.9113,42.5958,41.5926,32.871,34.0791,35.5563,41.7305,11.7187,7.01747,16.8824
std_fit_time,1.39212,1.13909,0.823052,1.5352,1.71904,0.962681,1.05662,1.00097,1.33455,0.800388,0.822679,0.731499
mean_score_time,1.66284,1.69212,1.76094,1.75943,1.58577,1.62931,1.58302,1.69387,1.7254,1.60379,1.29524,1.54148
std_score_time,0.694144,0.645778,0.527147,0.673374,0.847908,0.699741,0.784133,0.587094,0.686844,0.641421,0.340845,0.700728
param_randomforestclassifier__max_depth,20,20,None,None,None,15,15,15,15,10,5,5
param_randomforestclassifier__max_features,0.359094,0.568262,0.341815,0.797831,0.791825,0.742582,0.750058,0.810176,0.963716,0.311294,0.30606,0.890139
params,"{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand..."
split0_test_score,0.808632,0.807824,0.808767,0.797993,0.797993,0.8054,0.803313,0.802774,0.801293,0.775638,0.719345,0.715103
split1_test_score,0.807879,0.807071,0.806465,0.799327,0.799327,0.79468,0.794343,0.79468,0.793939,0.762155,0.718114,0.715286
split2_test_score,0.803569,0.802761,0.803232,0.796431,0.796431,0.791178,0.792593,0.790303,0.78963,0.757508,0.717441,0.715556


In [64]:
from xgboost.sklearn import XGBClassifier
from scipy.stats import beta, expon

one_to_left = beta(10, 1)
from_zero_positive = expon(0, 50)

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median'), 
    XGBClassifier(objective='multi:softprob', eval_metric='merror', nthread=-1)
)

param_distributions = {
    'xgbclassifier__n_estimators': randint(3, 40),
    'xgbclassifier__max_depth': randint(3, 40),
    'xgbclassifier__learning_rate': uniform(0.05, 0.4),
    'xgbclassifier__colsample_bytree': one_to_left,
    'xgbclassifier__subsample': one_to_left,
    'xgbclassifier__gamma': uniform(0, 10),
    'xgbclassifier__reg_alpha': from_zero_positive,
    'xgbclassifier__min_child_weight': from_zero_positive,
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3, 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   57.9s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:  1.4min remaining:    9.4s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.5min finished


In [65]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation accuracy', search.best_score_)
print(pipeline.named_steps['xgbclassifier'])

Best hyperparameters {'xgbclassifier__colsample_bytree': 0.9367971484384465, 'xgbclassifier__gamma': 0.04888485599864767, 'xgbclassifier__learning_rate': 0.2601950311802157, 'xgbclassifier__max_depth': 34, 'xgbclassifier__min_child_weight': 6.9125727604829805, 'xgbclassifier__n_estimators': 10, 'xgbclassifier__reg_alpha': 28.733201532063646, 'xgbclassifier__subsample': 0.9035768621978747}
Cross-validation accuracy 0.7658922558922558
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='merror',
              gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=-1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [61]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,0,1,8,4,2,9,6,3,5,7
mean_fit_time,35.114,19.5577,16.9116,9.76812,20.6127,6.39435,12.8468,12.3663,14.1872,4.2012
std_fit_time,0.566964,0.821487,0.354186,0.270066,0.641749,0.154287,1.33132,0.347577,0.179544,0.569754
mean_score_time,0.686532,0.491311,0.491223,0.441756,0.555725,0.435918,0.509009,0.498975,0.443936,0.432845
std_score_time,0.0884732,0.00237997,0.0294786,0.037969,0.107733,0.0324571,0.00305079,0.108259,0.0614189,0.0606944
param_xgbclassifier__colsample_bytree,0.966022,0.970406,0.982228,0.72808,0.994519,0.811157,0.991252,0.951639,0.983403,0.996686
param_xgbclassifier__gamma,2.24077,1.61406,0.792016,2.61111,2.54259,0.677223,3.55831,8.83217,7.19972,6.3059
param_xgbclassifier__learning_rate,0.249137,0.31806,0.368467,0.446983,0.20928,0.431475,0.212834,0.135713,0.37145,0.384017
param_xgbclassifier__max_depth,27,30,36,16,17,22,25,23,18,5
param_xgbclassifier__min_child_weight,1.92107,61.6062,51.5412,40.8303,35.2717,155.662,1.62854,16.596,19.6303,38.7473
param_xgbclassifier__n_estimators,33,33,28,15,36,15,10,15,22,10
